NetCDF to Parquet

In [2]:
import os
import xarray as xr
import pandas as pd
from tqdm import tqdm

# Define the root dir where the NetCDF files are located
root_dir = r"Z:\Thesis\Data\Met\ERA5"

# Define the years to process
years = [str(year) for year in range(2013, 2014)]

# Define the list of countries
countries = [
    #"Bahrain",
    #"Kuwait",
    #"Oman",
    #"Qatar",
    "Saudi Arabia",
    #"United Arab Emirates",
    #"Yemen"
]
# Define the subfolders to process
subfolders = ["surface",
              #"pressure"
             ]

# Define the output root dir for parquet files
output_root_dir = r"Z:\Thesis\Data\Met\ERA5_parquet_test"
os.makedirs(output_root_dir, exist_ok=True)

# Function to adjust country names for output folder
def adjust_country_name(country):
    if country == "Saudi Arabia":
        return "Saudi_Arabia"
    elif country == "United Arab Emirates":
        return "United_Arab_Emirates"
    return country

# Collect all NetCDF files into a list
netcdf_files = []

# List to store skipped files and error messages
skipped_files = []

# Iterate through the years
for year in years:
    year_dir = os.path.join(root_dir, year)
    for country in countries:
        country_dir = os.path.join(year_dir, country)
        for subfolder in subfolders:
            subfolder_dir = os.path.join(country_dir, subfolder)
            if not os.path.exists(subfolder_dir):
                continue
            files_in_subfolder = [
                os.path.join(subfolder_dir, f) for f in os.listdir(subfolder_dir) if f.endswith('.nc')
            ]
            netcdf_files.extend(files_in_subfolder)

# Process the files with a progress bar
for netcdf_path in tqdm(netcdf_files, desc="Processing files"):
    try:
        # Print the file being processed
        tqdm.write(f"Processing file: {netcdf_path}")
        
        # Read the NetCDF file using xarray
        ds = xr.open_dataset(netcdf_path)
        
        # Convert xarray dataset to pandas DataFrame
        df = ds.to_dataframe().reset_index()
        
        # Check for 'level' or 'pressure_level' and standardize the column name to 'level'
        if 'pressure_level' in df.columns:
            df.rename(columns={'pressure_level': 'level'}, inplace=True)
            tqdm.write(f"Renamed 'pressure_level' to 'level' for file: {netcdf_path}")
        elif 'level' not in df.columns:
            # If neither column exists, handle surface or raise an error
            if 'surface' in os.path.normpath(netcdf_path).split(os.sep):
                df['level'] = 1013.0  # Add a 'level' column for surface files
                tqdm.write(f"Added 'level' column with value 1013.0 for surface file: {netcdf_path}")
            else:
                raise ValueError("'level' column is missing in the pressure file")
        
        # Verify all levels are present for pressure files
        expected_levels = [1000, 925, 850, 700, 500, 300, 200, 100, 50, 10]
        if 'level' in df.columns:
            missing_levels = set(expected_levels) - set(df['level'].unique())
            if missing_levels:
                tqdm.write(f"Warning: Missing levels in {netcdf_path}: {missing_levels}")
        
        # Dynamically handle 'time' or 'valid_time'
        time_column = 'time' if 'time' in df.columns else 'valid_time' if 'valid_time' in df.columns else None
        if not time_column:
            raise ValueError("No 'time' or 'valid_time' column found")
        
        # Adjust the country name for output folder
        relative_path = os.path.relpath(netcdf_path, root_dir)
        parts = relative_path.split(os.sep)
        if len(parts) >= 2:
            parts[1] = adjust_country_name(parts[1])  # Adjust country name if needed
        adjusted_relative_path = os.path.join(*parts)
        
        # Adjust the filename explicitly for Saudi Arabia and United Arab Emirates
        country_name = os.path.basename(os.path.dirname(netcdf_path)).strip()
        original_filename = os.path.basename(netcdf_path).replace('.nc', '')
        if "Saudi Arabia" in original_filename:
            parquet_file = original_filename.replace("Saudi Arabia", "Saudi_Arabia") + '.parquet'
        elif "United Arab Emirates" in original_filename:
            parquet_file = original_filename.replace("United Arab Emirates", "United_Arab_Emirates") + '.parquet'
        else:
            parquet_file = original_filename + '.parquet'
        
        # Define the output directory and file path
        output_dir = os.path.join(output_root_dir, os.path.dirname(adjusted_relative_path))
        os.makedirs(output_dir, exist_ok=True)
        parquet_path = os.path.join(output_dir, parquet_file)
        
        # Write the DataFrame to a Parquet file
        df.to_parquet(parquet_path, index=False)
        tqdm.write(f"Successfully converted {netcdf_path} to {parquet_path}")
    
    except Exception as e:
        # Add the file and error to the skipped files list
        skipped_files.append((netcdf_path, str(e)))
        tqdm.write(f"Error processing {netcdf_path}: {e}")
        continue

# Print the summary of skipped files
if skipped_files:
    print("\nSummary of Skipped Files:")
    for file, error in skipped_files:
        print(f"{file}: {error}")

    # Optionally save the skipped files to a CSV for further investigation
    skipped_df = pd.DataFrame(skipped_files, columns=["File Path", "Error Message"])
    skipped_df.to_csv(os.path.join(output_root_dir, "netCDF2Parquet_skipped_files_13_sfc.csv"), index=False)
    print(f"Skipped files list saved to {os.path.join(output_root_dir, 'skipped_files.csv')}")
else:
    print("\nAll files processed successfully!")


Processing files:   0%|                                      | 0/26 [00:00<?, ?it/s]

Processing file: Z:\Thesis\Data\Met\ERA5\2013\Saudi Arabia\surface\2013_Saudi Arabia_surface_10m_u_component_of_wind.nc


Processing files:   0%|                                      | 0/26 [00:11<?, ?it/s]

Added 'level' column with value 1013.0 for surface file: Z:\Thesis\Data\Met\ERA5\2013\Saudi Arabia\surface\2013_Saudi Arabia_surface_10m_u_component_of_wind.nc


Processing files:   0%|                                      | 0/26 [00:12<?, ?it/s]

...

Successfully converted Z:\Thesis\Data\Met\ERA5\2013\Saudi Arabia\surface\2013_Saudi Arabia_surface_volumetric_soil_water_layer_3.nc to Z:\Thesis\Data\Met\ERA5_parquet_test\2013\Saudi_Arabia\surface\2013_Saudi_Arabia_surface_volumetric_soil_water_layer_3.parquet
Processing file: Z:\Thesis\Data\Met\ERA5\2013\Saudi Arabia\surface\2013_Saudi Arabia_surface_volumetric_soil_water_layer_4.nc


Processing files:  96%|███████████████████████████▉ | 25/26 [03:34<00:08,  8.40s/it]

Added 'level' column with value 1013.0 for surface file: Z:\Thesis\Data\Met\ERA5\2013\Saudi Arabia\surface\2013_Saudi Arabia_surface_volumetric_soil_water_layer_4.nc


Processing files: 100%|█████████████████████████████| 26/26 [03:38<00:00,  8.41s/it]

Successfully converted Z:\Thesis\Data\Met\ERA5\2013\Saudi Arabia\surface\2013_Saudi Arabia_surface_volumetric_soil_water_layer_4.nc to Z:\Thesis\Data\Met\ERA5_parquet_test\2013\Saudi_Arabia\surface\2013_Saudi_Arabia_surface_volumetric_soil_water_layer_4.parquet

All files processed successfully!


Convert Valid Time to Time

In [10]:
import os
import pandas as pd

# Define the base directory and the range of years
base_dir = r"Z:\Thesis\Data\Met\ERA5_parquet_test"
years = range(1989, 1990)  # 1989 to 2000 inclusive

# Define the countries dictionary
countries_dict = {
    "SA": "Saudi_Arabia",
    #"YE": "Yemen",
    #"OM": "Oman",
    #"QA": "Qatar",
    #"BH": "Bahrain",
    #"AE": "United_Arab_Emirates",
    #"KW": "Kuwait"
}

# Define subdirectories to iterate through
folders = [
    "surface", 
    "pressure"
          ]

# Create a list to keep track of files that were skipped due to errors
skipped_files = []

# Define an output directory for the CSV summary (you can change this as needed)
output_root_dir = base_dir  # For example, using base_dir here

# Iterate over years, countries, and folders
for year in years:
    for code, country in countries_dict.items():
        for folder in folders:
            folder_path = os.path.join(base_dir, str(year), country, folder)
            
            # Check if the folder exists
            if os.path.exists(folder_path):
                for file in os.listdir(folder_path):
                    # Process only parquet files
                    if file.endswith(".parquet"):
                        file_path = os.path.join(folder_path, file)
                        
                        try:
                            # Load the parquet file
                            df = pd.read_parquet(file_path)
                            
                            # Check if 'valid_time' column exists and rename it to 'time'
                            if 'valid_time' in df.columns:
                                df.rename(columns={'valid_time': 'time'}, inplace=True)
                                
                                # Save the updated dataframe back to the parquet file
                                df.to_parquet(file_path, index=False)
                                print(f"Updated column in file: {file_path}")
                        except Exception as e:
                            # Print error message and log the file as skipped
                            print(f"Error processing file {file_path}: {e}")
                            skipped_files.append((file_path, str(e)))
            else:
                print(f"Folder does not exist: {folder_path}")

# After processing, check and report any skipped files
if skipped_files:
    print("\nSummary of Skipped Files:")
    for file, error in skipped_files:
        print(f"{file}: {error}")

    # Optionally, save the skipped files information to a CSV for further investigation
    skipped_df = pd.DataFrame(skipped_files, columns=["File Path", "Error Message"])
    csv_path = os.path.join(output_root_dir, "time_skipped_files.csv")
    skipped_df.to_csv(csv_path, index=False)
    print(f"Skipped files list saved to {csv_path}")
else:
    print("\nAll files processed successfully!")



All files processed successfully!


ERA5 Monthly Statistics Calculation 

In [13]:
import os
import pandas as pd
from pathlib import Path

# Define the years to process
years = [str(year) for year in range(1989, 2001)]

# Dictionaries for surface and pressure variables
surface_file_naming = {
    "2m_temperature": "2m_temperature",
    "2m_dewpoint_temperature": "2m_dewpoint_temperature",
    "10m_u_component_of_wind": "10m_u_component_of_wind",
    "10m_v_component_of_wind": "10m_v_component_of_wind",
    "surface_solar_radiation_downwards": "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards": "surface_thermal_radiation_downwards",
    "evaporation": "evaporation",
    "potential_evaporation": "potential_evaporation",
    "sea_surface_temperature": "sea_surface_temperature",
    "soil_temperature_level_1": "soil_temperature_level_1",
    "soil_temperature_level_2": "soil_temperature_level_2",
    "soil_temperature_level_3": "soil_temperature_level_3",
    "soil_temperature_level_4": "soil_temperature_level_4",
    "volumetric_soil_water_layer_1": "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2": "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3": "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4": "volumetric_soil_water_layer_4",
    "leaf_area_index_high_vegetation": "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation": "leaf_area_index_low_vegetation",
    "surface_pressure": "surface_pressure",
    "mean_sea_level_pressure": "mean_sea_level_pressure",
    "convective_available_potential_energy": "convective_available_potential_energy",
    "geopotential": "geopotential",
    "instantaneous_10m_wind_gust": "instantaneous_10m_wind_gust",
    "total_precipitation": "total_precipitation",
    "k_index": "k_index",
}

pressure_file_naming = {
    "geopotential": "geopotential",
    "relative_humidity": "relative_humidity",
    "temperature": "temperature",
    "u_component_of_wind": "u_component_of_wind",
    "v_component_of_wind": "v_component_of_wind",
    "vertical_velocity": "vertical_velocity",
    "vorticity": "vorticity",
}

# Variable abbreviations for accessing data
variable_abbreviations = {
    "2m_temperature": "t2m",
    "2m_dewpoint_temperature": "d2m",
    "10m_u_component_of_wind": "u10",
    "10m_v_component_of_wind": "v10",
    "surface_solar_radiation_downwards": "ssrd",
    "surface_thermal_radiation_downwards": "strd",
    "evaporation": "e",
    "potential_evaporation": "pev",
    "sea_surface_temperature": "sst",
    "soil_temperature_level_1": "stl1",
    "soil_temperature_level_2": "stl2",
    "soil_temperature_level_3": "stl3",
    "soil_temperature_level_4": "stl4",
    "volumetric_soil_water_layer_1": "swvl1",
    "volumetric_soil_water_layer_2": "swvl2",
    "volumetric_soil_water_layer_3": "swvl3",
    "volumetric_soil_water_layer_4": "swvl4",
    "leaf_area_index_high_vegetation": "lai_hv",
    "leaf_area_index_low_vegetation": "lai_lv",
    "surface_pressure": "sp",
    "mean_sea_level_pressure": "msl",
    "convective_available_potential_energy": "cape",
    "geopotential": "z",
    "instantaneous_10m_wind_gust": "i10fg",
    "total_precipitation": "tp",
    "k_index": "kx",
    "relative_humidity": "r",
    "temperature": "t",
    "u_component_of_wind": "u",
    "v_component_of_wind": "v",
    "vertical_velocity": "w",
    "vorticity": "vo",
}

# File paths
BASE_PATH = r"Z:\Thesis\Data\Met\ERA5_parquet_test"
OUTPUT_PATH = r"Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats"

def calculate_monthly_statistics(year, country, surface_file_naming, pressure_file_naming, variable_abbreviations):
    """
    Calculate monthly statistics for all variables and save the results as separate Parquet files for surface and pressure data.
    """
    country_path = os.path.join(BASE_PATH, year, country)
    output_dir = os.path.join(OUTPUT_PATH, year)
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Separate results for surface and pressure data
    surface_stats_list = []
    pressure_stats_list = []
    missing_files = []

    def process_file(file, column_name, include_level=False):
        """
        Process an individual file, calculate monthly statistics, and append to the results list.
        """
        df = pd.read_parquet(file)

        # Define grouping columns based on whether `level` should be included
        group_by_columns = ["longitude", "latitude", pd.Grouper(key="time", freq="ME")]
        if include_level and "level" in df.columns:
            group_by_columns.append("level")

        # Group by month and calculate statistics
        grouped = df.groupby(group_by_columns)
        monthly_stats = grouped.agg({
            column_name: ["min", "max", "mean"]
        }).reset_index()

        # Flatten column names
        monthly_stats.columns = [
            col if isinstance(col, str) else "_".join(col).strip("_")
            for col in monthly_stats.columns
        ]
        return monthly_stats

    # Process surface variables (no dynamic levels)
    for variable, file_name in surface_file_naming.items():
        abbreviation = variable_abbreviations[variable]
        file_path = os.path.join(country_path, "surface", f"{year}_{country}_surface_{file_name}.parquet")
        if os.path.exists(file_path):
            surface_stats_list.append(process_file(file_path, abbreviation, include_level=True))
        else:
            missing_files.append(file_path)

    # Process pressure variables (include dynamic levels)
    for variable, file_name in pressure_file_naming.items():
        abbreviation = variable_abbreviations[variable]
        file_path = os.path.join(country_path, "pressure", f"{year}_{country}_pressure_{file_name}.parquet")
        if os.path.exists(file_path):
            pressure_stats_list.append(process_file(file_path, abbreviation, include_level=True))
        else:
            missing_files.append(file_path)

    # Save surface data
    if surface_stats_list:
        combined_surface_stats = pd.concat(surface_stats_list, axis=1)
        combined_surface_stats = combined_surface_stats.loc[:, ~combined_surface_stats.columns.duplicated()]  # Remove duplicate columns
        surface_output_file = os.path.join(output_dir, f"{country}_{year}_surface_monthly_stats.parquet")
        combined_surface_stats.to_parquet(surface_output_file, index=False)
        print(f"Saved surface monthly statistics for {country} in {year} to {surface_output_file}")
    else:
        print(f"No surface data processed for {country} in {year}.")
    
    # Save pressure data
    if pressure_stats_list:
        combined_pressure_stats = pd.concat(pressure_stats_list, axis=1)
        combined_pressure_stats = combined_pressure_stats.loc[:, ~combined_pressure_stats.columns.duplicated()]  # Remove duplicate columns
        pressure_output_file = os.path.join(output_dir, f"{country}_{year}_pressure_monthly_stats.parquet")
        combined_pressure_stats.to_parquet(pressure_output_file, index=False)
        print(f"Saved pressure monthly statistics for {country} in {year} to {pressure_output_file}")
    else:
        print(f"No pressure data processed for {country} in {year}.")

    # Log missing files
    if missing_files:
        print(f"Missing files for {country} in {year}:")
        for file in missing_files:
            print(f"  {file}")



if __name__ == "__main__":
    # List of countries to process
    countries = [#"Bahrain",
                 "Saudi_Arabia",
                 #"Oman",
                 #"Qatar",
                 #"United_Arab_Emirates",
                 #"Kuwait",
                 #"Yemen"
                ]

    for year in years:
        for country in countries:
            calculate_monthly_statistics(year, country, surface_file_naming, pressure_file_naming, variable_abbreviations)


Saved surface monthly statistics for Saudi_Arabia in 1989 to Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Saudi_Arabia_1989_surface_monthly_stats.parquet
Saved pressure monthly statistics for Saudi_Arabia in 1989 to Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Saudi_Arabia_1989_pressure_monthly_stats.parquet
Saved surface monthly statistics for Saudi_Arabia in 1990 to Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1990\Saudi_Arabia_1990_surface_monthly_stats.parquet
Saved pressure monthly statistics for Saudi_Arabia in 1990 to Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1990\Saudi_Arabia_1990_pressure_monthly_stats.parquet
Saved surface monthly statistics for Saudi_Arabia in 1991 to Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1991\Saudi_Arabia_1991_surface_monthly_stats.parquet
Saved pressure monthly statistics for Saudi_Arabia in 1991 to Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1991\Saudi_Arabia_1991_pressure_monthly_stats.parquet
Save

Add H3 Resolution 4 index

In [1]:
import os
import pandas as pd
import h3

# Define the year range
start_year = 1989
end_year = 2001  # inclusive

# Countries for ERA5 data
countries = [
    "Saudi_Arabia",
    #"United_Arab_Emirates",
    #"Yemen",
    #"Bahrain",
    #"Kuwait",
    #"Oman",
    #"Qatar"
]

# Base directories
merra2_base = r"Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats"
era5_base = r"Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats"

# Compile a list of file paths
file_paths = []

for year in range(start_year, end_year+1):
    # MERRA2 path for the given year
    merra2_path = os.path.join(merra2_base, str(year), f"MERRA2_{year}_monthly_stats.parquet")
    file_paths.append(merra2_path)

    # ERA5 paths for the given year and each country
    for country in countries:
        pressure_path = os.path.join(era5_base, str(year), f"{country}_{year}_pressure_monthly_stats.parquet")
        surface_path = os.path.join(era5_base, str(year), f"{country}_{year}_surface_monthly_stats.parquet")
        file_paths.append(pressure_path)
        file_paths.append(surface_path)

# Define which H3 resolutions we want
h3_resolutions = [4]

for file_path in file_paths:
    if not os.path.exists(file_path):
        print(f"File does not exist, skipping: {file_path}")
        continue

    print(f"Processing {file_path}")

    # Load the parquet file into a DataFrame
    df = pd.read_parquet(file_path)

    # Determine if file is MERRA2 or ERA5 based on file path
    if "MERRA2" in file_path:
        # MERRA2 uses 'lat' and 'lon'
        # Check if the columns exist as expected
        if 'lat' not in df.columns or 'lon' not in df.columns:
            print(f"lat/lon columns not found for MERRA2 file: {file_path}, skipping.")
            continue
    else:
        # ERA5 uses 'latitude' and 'longitude', so we rename them to 'lat' and 'lon'
        if 'latitude' in df.columns and 'longitude' in df.columns:
            df.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)
        # After attempted rename, check if 'lat' and 'lon' now exist
        if 'lat' not in df.columns or 'lon' not in df.columns:
            print(f"lat/lon columns not found for ERA5 file: {file_path}, skipping.")
            continue


    # Compute H3 indices at desired resolutions
    for res in h3_resolutions:
        h3_col_name = f"h3_res_{res}"
        # Compute the H3 string index
        df[h3_col_name] = df.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], res), axis=1)
        
        # Convert the H3 string to a numeric index
        h3_int_col = f"h3_res_{res}_int"
        df[h3_int_col] = df[h3_col_name].apply(lambda x: int(x, 16))

    # Save the updated DataFrame back to parquet (overwriting)
    df.to_parquet(file_path, index=False)
    print(f"Updated file saved: {file_path}")


Processing Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\1989\MERRA2_1989_monthly_stats.parquet
Updated file saved: Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\1989\MERRA2_1989_monthly_stats.parquet
Processing Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Saudi_Arabia_1989_pressure_monthly_stats.parquet
Updated file saved: Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Saudi_Arabia_1989_pressure_monthly_stats.parquet
Processing Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Saudi_Arabia_1989_surface_monthly_stats.parquet
Updated file saved: Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Saudi_Arabia_1989_surface_monthly_stats.parquet
Processing Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\1990\MERRA2_1990_monthly_stats.parquet
Updated file saved: Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\1990\MERRA2_1990_monthly_stats.parquet
Processing Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\

Join ERA5 & MERRA-2 Data

In [2]:
import pandas as pd
import os

# Base paths
era5_base_path = r"Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats"
merra2_base_path = r"Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats"
output_base_path = r"Z:\Thesis\Data\ML_Data\AP_training2"

# Columns to merge for surface and pressure data
surface_aod_cols = [
    "DUSMASS_min", "DUSMASS_max", "DUSMASS_mean",
    "DUSMASS25_min", "DUSMASS25_max", "DUSMASS25_mean",
    "DUFLUXU_min", "DUFLUXU_max", "DUFLUXU_mean",
    "DUFLUXV_min", "DUFLUXV_max", "DUFLUXV_mean"
]

pressure_aod_cols = [
    "DUCMASS_min", "DUCMASS_max", "DUCMASS_mean",
    "DUCMASS25_min", "DUCMASS25_max", "DUCMASS25_mean",
    "DUFLUXU_min", "DUFLUXU_max", "DUFLUXU_mean",
    "DUFLUXV_min", "DUFLUXV_max", "DUFLUXV_mean"
]

# Process each year
for year in range(1989, 2001):
    print(f"Processing year: {year}")

    # Paths for the current year
    merra2_file = os.path.join(merra2_base_path, str(year), f"MERRA2_{year}_monthly_stats.parquet")
    era5_year_path = os.path.join(era5_base_path, str(year))

    if not os.path.exists(merra2_file):
        print(f"Skipping year {year}: MERRA2 file not found.")
        continue
    if not os.path.exists(era5_year_path):
        print(f"Skipping year {year}: ERA5 directory not found.")
        continue

    # Load MERRA2 data
    try:
        merra2_df = pd.read_parquet(merra2_file)
        print(f"Loaded MERRA2 data: {merra2_file}")
        if 'time' not in merra2_df.columns:
            print(f"⚠️ WARNING: 'time' column missing in {merra2_file}")
            continue
        merra2_df['time'] = pd.to_datetime(merra2_df['time'])
    except Exception as e:
        print(f"❌ ERROR reading MERRA2 file {merra2_file}: {e}")
        continue

    # Process each ERA5 file in the year's directory
    for era5_file in os.listdir(era5_year_path):
        era5_file_path = os.path.join(era5_year_path, era5_file)

        # Determine if this file is surface or pressure based on its name
        filename = os.path.basename(era5_file_path).lower()
        if "surface" in filename:
            aod_cols = surface_aod_cols
        elif "pressure" in filename:
            aod_cols = pressure_aod_cols
        else:
            print(f"Skipping {era5_file_path}: Unable to determine type.")
            continue

        # Read ERA5 data
        try:
            era5_df = pd.read_parquet(era5_file_path)
            print(f"✅ Successfully loaded: {era5_file_path}")

            if 'time' not in era5_df.columns:
                print(f"⚠️ WARNING: 'time' column missing in {era5_file_path}. Skipping file.")
                continue

            era5_df['time'] = pd.to_datetime(era5_df['time'])

        except Exception as e:
            print(f"❌ ERROR reading {era5_file_path}: {e}")
            continue

        # Join MERRA2 and ERA5 data
        join_keys = ['time', 'h3_res_4']
        if not all(col in merra2_df.columns for col in join_keys):
            print(f"⚠️ WARNING: One or more join keys missing in MERRA2 data for {year}. Skipping.")
            continue

        try:
            merra2_subset = merra2_df[join_keys + aod_cols]
            merged_df = pd.merge(era5_df, merra2_subset, on=join_keys, how='left')

            # Drop rows that do not have a match
            merged_df = merged_df.dropna(subset=aod_cols)

            # Save the merged output
            output_dir = os.path.join(output_base_path, str(year))
            os.makedirs(output_dir, exist_ok=True)
            output_file = os.path.join(output_dir, os.path.splitext(era5_file)[0] + "_merged.parquet")
            merged_df.to_parquet(output_file, index=False)
            print(f"💾 Saved merged file: {output_file}")

        except Exception as e:
            print(f"❌ ERROR merging data for {era5_file_path}: {e}")
            continue


Processing year: 1989
Loaded MERRA2 data: Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\1989\MERRA2_1989_monthly_stats.parquet
✅ Successfully loaded: Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Bahrain_1989_surface_monthly_stats.parquet
💾 Saved merged file: Z:\Thesis\Data\ML_Data\AP_training2\1989\Bahrain_1989_surface_monthly_stats_merged.parquet
✅ Successfully loaded: Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Bahrain_1989_pressure_monthly_stats.parquet
💾 Saved merged file: Z:\Thesis\Data\ML_Data\AP_training2\1989\Bahrain_1989_pressure_monthly_stats_merged.parquet
✅ Successfully loaded: Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Oman_1989_surface_monthly_stats.parquet
💾 Saved merged file: Z:\Thesis\Data\ML_Data\AP_training2\1989\Oman_1989_surface_monthly_stats_merged.parquet
✅ Successfully loaded: Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats\1989\Oman_1989_pressure_monthly_stats.parquet
💾 Saved merged file: Z:\Thesis\Data\ML_Da

Join IOD Data to ERA5 & MERRA2 Data

In [4]:
import pandas as pd
import os
from glob import glob

# Paths to DMI CSV files
dmi_csv_files = {
    "DMI EAST HadISST1.1": "Z:\\Thesis\\Data\\Met\\DMI\\dmieast.had.long.csv",
    "DMI HadISST1.1": "Z:\\Thesis\\Data\\Met\\DMI\\dmi.had.long.csv"
}

# Paths to parquet files
data_directory = "Z:\\Thesis\\Data\\ML_Data\\AP_training2"
output_directory = "Z:\\Thesis\\Data\\ML_Data\\AP_training2"
years = list(range(1989, 2001)) + list(range(2013, 2016))

# Read and preprocess DMI data
def load_dmi_data(file_paths):
    dmi_frames = []
    for column_name, file_path in file_paths.items():
        dmi_data = pd.read_csv(file_path)
        # Parse dates and set them to the last day of the month
        dmi_data['Date'] = pd.to_datetime(dmi_data['Date'], format='%m/%d/%Y', errors='coerce')
        dmi_data['Date'] = dmi_data['Date'] + pd.offsets.MonthEnd(0)
        # Rename the value column to match the key for clarity
        dmi_data = dmi_data[['Date', column_name]]
        dmi_frames.append(dmi_data)
    combined_dmi_data = pd.concat(dmi_frames, axis=1)
    # Remove duplicate 'Date' columns, keeping only one
    combined_dmi_data = combined_dmi_data.loc[:, ~combined_dmi_data.columns.duplicated()]
    return combined_dmi_data

dmi_data = load_dmi_data(dmi_csv_files)

def process_and_merge_parquet_files(base_dir, years, dmi_data, output_dir):
    for year in years:
        year_path = os.path.join(base_dir, str(year))
        parquet_files = glob(os.path.join(year_path, "*.parquet"))

        for parquet_file in parquet_files:
            # Load parquet file
            parquet_data = pd.read_parquet(parquet_file)

            # Ensure the 'time' column is in datetime format and set to the last day of the month
            parquet_data['time'] = pd.to_datetime(parquet_data['time'], errors='coerce')
            parquet_data['time'] = parquet_data['time'] + pd.offsets.MonthEnd(0)

            # Merge with DMI data
            merged_data = parquet_data.merge(
                dmi_data,
                left_on='time',
                right_on='Date',
                how='left'
            )

            # Drop unnecessary 'Date' column from the output
            merged_data = merged_data.drop(columns=['Date'])

            # Save merged data to a new parquet file in the output directory
            relative_path = os.path.relpath(parquet_file, base_dir)
            output_file = os.path.join(output_dir, relative_path)
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            merged_data.to_parquet(output_file, index=False)

            print(f"Merged data saved to {output_file}")

# Run the merging process
process_and_merge_parquet_files(data_directory, years, dmi_data, output_directory)


Merged data saved to Z:\Thesis\Data\ML_Data\AP_training2\1989\Bahrain_1989_surface_monthly_stats_merged.parquet
Merged data saved to Z:\Thesis\Data\ML_Data\AP_training2\1989\Bahrain_1989_pressure_monthly_stats_merged.parquet
Merged data saved to Z:\Thesis\Data\ML_Data\AP_training2\1989\Oman_1989_surface_monthly_stats_merged.parquet
Merged data saved to Z:\Thesis\Data\ML_Data\AP_training2\1989\Oman_1989_pressure_monthly_stats_merged.parquet
Merged data saved to Z:\Thesis\Data\ML_Data\AP_training2\1989\Qatar_1989_surface_monthly_stats_merged.parquet
Merged data saved to Z:\Thesis\Data\ML_Data\AP_training2\1989\Qatar_1989_pressure_monthly_stats_merged.parquet
Merged data saved to Z:\Thesis\Data\ML_Data\AP_training2\1989\United_Arab_Emirates_1989_surface_monthly_stats_merged.parquet
Merged data saved to Z:\Thesis\Data\ML_Data\AP_training2\1989\United_Arab_Emirates_1989_pressure_monthly_stats_merged.parquet
Merged data saved to Z:\Thesis\Data\ML_Data\AP_training2\1989\Kuwait_1989_surface_mo